In [1]:
import pickle
import pandas as pd
from msresist.clustering import DDMC
from msresist.pre_processing import filter_NaNpeptides

In [2]:
X = pd.read_csv("msresist/data/MS/CPTAC/CPTAC-preprocessedMotfis.csv").iloc[:, 1:]
X = filter_NaNpeptides(X, tmt=2)
d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

In [3]:
model = DDMC(i, ncl=5, SeqWeight=0, distance_method="Binomial").fit(d, "NA", nRepeats=0)

PCA done
[[ 0.50077482 -0.78100496 -1.15893376 ... -0.28323929  0.79271239
  -0.38408118]
 [-0.11503311  0.22531788 -0.85941641 ...  0.19979948 -0.19577283
  -0.02668093]
 [ 0.49562879 -1.10790224 -0.20144384 ... -0.80597433  0.64544475
  -0.67206968]
 [ 0.38538329 -0.47284583 -0.36852425 ... -0.28163436  0.39991024
  -0.29310801]
 [ 0.09862686 -2.13696331  0.01299713 ... -1.65977931  0.5456944
  -1.34881141]]
[1] Improvement: 87084.0360210163	Time (s): 0.7154
Total Improvement: 87084.0360210163
Total Time (s): 1.0548


In [4]:
model.transform()

array([[ 0.33183607,  0.33143113,  0.33214434,  0.33150871,  0.3312932 ],
       [-0.73208109, -0.7324785 , -0.73178168, -0.73239767, -0.73261282],
       [-0.52272892, -0.52081181, -0.52417305, -0.52117689, -0.52013892],
       ...,
       [-0.43666258, -0.43685183, -0.43651498, -0.4368112 , -0.43691311],
       [ 0.47764469,  0.47781007,  0.47753022,  0.477779  ,  0.47786268],
       [-0.45133698, -0.45118702, -0.45144683, -0.45120823, -0.45112632]])

In [15]:
# with open("msresist/data/pickled_models/binomial/CPTACmodel_binomial_CL20_W20", "rb") as w:
#     model = pickle.load(w)[0]

In [1]:
# pd.DataFrame(model.transform())